In [231]:
import io
import pathlib

import requests

from PIL import Image
from multiprocessing.pool import ThreadPool


def download_imgs(img_urls, download_path):
    download_path.mkdir(exist_ok=True)

    for img_url in img_urls:
        print(f'Downloading: {img_url}')
        res = requests.get(img_url, stream=True)
        count = 1
        while res.status_code != 200 and count <= 5:
            res = requests.get(img_url, stream=True)
            print(f'Retry: {count} {img_url}')
            count += 1
        try:
            image_name = str(img_url[(img_url.rfind('/')) + 1:])
            if '?' in image_name:
                image_name = image_name[:image_name.find('?')]
        except:
            image_name = str(random.randint(11111, 99999))+'.jpg'
        
        if "." not in image_name:
            image_name = image_name + ".jpg"
        im = Image.open(io.BytesIO(res.content))
        if min(im.size) < 128: 
            continue
        else:
            im.save(download_path/image_name)

# results = ThreadPool(process).imap_unordered(image_downloader, images_url)

In [275]:
#1. Loading BeautifulSoup and test request
from bs4 import BeautifulSoup
import requests
import re

CIAN_URL = "https://www.cian.ru/cat.php?currency=2&engine_version=2&offer_type=flat"#&totime=86400"


def get_cian_search_results_count(url):
    cian_html = requests.get(url).text
    soup = BeautifulSoup(cian_html)
    counter_string = soup.find('div', {'data-name': 'SummaryHeader'}).text
    try:
        return int("".join(re.findall('\d+', counter_string)))
    except (ValueError, AttributeError) as e:
        return 0
       
def load_cian_flat_dicts(url):
    cian_html = requests.get(url).text
    soup = BeautifulSoup(cian_html)

    flats_dict = {}
    flats = soup.find_all('div', {'data-name': re.compile('OfferCard')})
    for f in flats:
        flat_imgs = []

        additional_imgs = f.find_all('img', {'data-name': 'GalleryImage'})
        if additional_imgs:
            flat_imgs.append(f.find('img')['src'])

        for fa in additional_imgs:
            flat_imgs.append(fa['src'])

        links = f.find_all('a', {'target': '_blank'})
        for a in links:
            if 'https://www.cian.ru/rent/flat/' or 'https://www.cian.ru/sale/flat/' in a['href']:
                if '/cat.php?' not in a['href'] and flat_imgs:
                    flats_dict[a['href']] = list(set(flat_imgs))
                    break

    return flats_dict

def get_cian_url(deal_type='rent', minprice=40000, maxprice=60000, number_of_rooms=1, region=1, page=1):
    return f"{CIAN_URL}&region={region}&deal_type={deal_type}&minprice={minprice}&maxprice={maxprice}&room1=1&p={page}"

In [241]:
AVITO_URL = "https://www.avito.ru/moskva/kvartiry/sdam/1-komnatnye-ASgBAQICAUSSA8gQAUDMCBSOWQ?cd=1&s=104&f=ASgBAQICAkSSA8gQ8AeQUgFAzAgUjlk"

def get_avito_url(minprice=40000, maxprice=60000, page=1):
    return f"{AVITO_URL}&pmin={minprice}&pmax={maxprice}&p={page}"


def get_avito_search_results_count(url):
    avito_html = requests.get(url).text
    soup = BeautifulSoup(avito_html)
    counter_string = soup.find('span', {"class": re.compile("^page-title-count*")}).text
    try:
        return int("".join(re.findall('\d+', counter_string)))
    except (ValueError, AttributeError) as e:
        return 0
    

def load_avito_flat_dicts(url):
    avito_html = requests.get(url).text
    soup = BeautifulSoup(avito_html)

    flats_dict = {}
    flats = soup.find_all('a', {'class': 'js-item-slider item-slider'})
        
    for f in flats:
        flat_imgs = []
        link = f['href']
        
        imgs = f.find_all('img', {'class': 'large-picture-img'})
        for img in imgs:
            if img.has_attr('srcset'):
                flat_imgs.append(img['srcset'].split()[-2])
            elif img.has_attr('data-srcset'):
                flat_imgs.append(img['data-srcset'].split()[-2])

        if 'kvartiry/' in link and flat_imgs:
            flats_dict[f"https://avito.ru{link}"] = list(set(flat_imgs))

    return flats_dict

CIAN URLS for training

In [276]:
PAGE_SIZE = 28

cian_url = get_cian_url(minprice=85000, maxprice=150000)

flats_dict_cian = {}
search_results_count = get_cian_search_results_count(cian_url)

print(f'There are {search_results_count} flats at your request. Grabbing the data...')

max_page_number = search_results_count // PAGE_SIZE
if not search_results_count % PAGE_SIZE or max_page_number == 0:
    max_page_number += 1

for page in tqdm(range(1, max_page_number + 1)):
    cian_url = get_cian_url(minprice=85000, maxprice=150000, page=page)
    flats_dict_cian.update(load_cian_flat_dicts(cian_url))

There are 207 flats at your request. Grabbing the data...


AVITO URLS for training

In [277]:
PAGE_SIZE = 48

flats_dict_avito = {}
avito_url = get_avito_url(minprice=85000, maxprice=150000)

search_results_count = get_avito_search_results_count(avito_url)

print(f'There are {search_results_count} flats at your request. Grabbing the data...')

max_page_number = search_results_count // PAGE_SIZE
if not search_results_count % PAGE_SIZE or max_page_number == 0:
    max_page_number += 1

for page in tqdm(range(1, max_page_number + 1)):
    url = get_avito_url(minprice=85000, maxprice=150000, page=page)
    flats_dict_avito.update(load_avito_flat_dicts(url))

There are 68 flats at your request. Grabbing the data...


In [281]:
download_path_cian = Path("data/cian_good/")
download_path_cian.mkdir(exist_ok=True)
for k, v in tqdm(list(flats_dict_cian.items())):
    download_imgs(v, download_path_cian/k.split("/")[-2])
    
download_path_avito = Path("data/avito_good/")
download_path_avito.mkdir(exist_ok=True)
for k, v in tqdm(list(flats_dict_avito.items())):
    try:
        download_imgs(v, download_path_avito/k.split("/")[-1].split(".")[-1][1:])
    except: 
        print('error with', k)
        continue

Downloading: https://cdn-p.cian.site/images/4/048/765/kvartira-moskva-shlyuzovaya-naberezhnaya-567840439-2.jpg
Downloading: https://cdn-p.cian.site/images/4/048/765/kvartira-moskva-shlyuzovaya-naberezhnaya-567840440-2.jpg
Downloading: https://cdn-p.cian.site/images/4/048/765/kvartira-moskva-shlyuzovaya-naberezhnaya-567840442-2.jpg
Downloading: https://cdn-p.cian.site/images/4/048/765/kvartira-moskva-shlyuzovaya-naberezhnaya-567840441-4.jpg
Downloading: https://cdn-p.cian.site/images/4/048/765/kvartira-moskva-shlyuzovaya-naberezhnaya-567840448-2.jpg
Downloading: https://cdn-p.cian.site/images/1/075/019/kvartira-moskva-1y-krasnogvardeyskiy-proezd-910570188-2.jpg
Downloading: https://cdn-p.cian.site/images/1/075/019/kvartira-moskva-1y-krasnogvardeyskiy-proezd-910570174-2.jpg
Downloading: https://cdn-p.cian.site/images/1/075/019/kvartira-moskva-1y-krasnogvardeyskiy-proezd-910570194-2.jpg
Downloading: https://cdn-p.cian.site/images/1/075/019/kvartira-moskva-1y-krasnogvardeyskiy-proezd-91057

Downloading: https://cdn-p.cian.site/images/9/554/809/kvartira-moskva-ulica-ordzhonikidze-908455969-2.jpg
Downloading: https://cdn-p.cian.site/images/7/554/809/kvartira-moskva-ulica-ordzhonikidze-908455749-4.jpg
Downloading: https://cdn-p.cian.site/images/3/944/509/905449343-2.jpg
Downloading: https://cdn-p.cian.site/images/8/750/609/906057866-2.jpg
Downloading: https://cdn-p.cian.site/images/3/944/509/905449317-4.jpg
Downloading: https://cdn-p.cian.site/images/3/944/509/905449347-2.jpg
Downloading: https://cdn-p.cian.site/images/3/944/509/905449356-2.jpg
Downloading: https://cdn-p.cian.site/images/2/605/309/903506212-2.jpg
Downloading: https://cdn-p.cian.site/images/2/605/309/903506202-2.jpg
Downloading: https://cdn-p.cian.site/images/9/741/709/907147966-2.jpg
Downloading: https://cdn-p.cian.site/images/2/605/309/903506209-2.jpg
Downloading: https://cdn-p.cian.site/images/1/605/309/903506193-4.jpg
Downloading: https://cdn-p.cian.site/images/0/909/709/907909067-2.jpg
Downloading: https

Downloading: https://cdn-p.cian.site/images/3/843/009/900348315-2.jpg
Downloading: https://cdn-p.cian.site/images/2/843/009/900348280-4.jpg
Downloading: https://cdn-p.cian.site/images/3/843/009/900348340-2.jpg
Downloading: https://cdn-p.cian.site/images/4/272/648/kvartira-moskva-pereulok-krasina-846272410-2.jpg
Downloading: https://cdn-p.cian.site/images/4/272/648/kvartira-moskva-pereulok-krasina-846272410-4.jpg
Downloading: https://cdn-p.cian.site/images/4/272/648/kvartira-moskva-pereulok-krasina-846272400-2.jpg
Downloading: https://cdn-p.cian.site/images/4/272/648/kvartira-moskva-pereulok-krasina-846272412-2.jpg
Downloading: https://cdn-p.cian.site/images/3/272/648/kvartira-moskva-pereulok-krasina-846272366-2.jpg
Downloading: https://cdn-p.cian.site/images/5/826/358/853628539-2.jpg
Downloading: https://cdn-p.cian.site/images/4/826/358/853628421-2.jpg
Downloading: https://cdn-p.cian.site/images/5/826/358/853628502-2.jpg
Downloading: https://cdn-p.cian.site/images/2/826/358/853628255-4

Downloading: https://cdn-p.cian.site/images/5/102/808/808201582-2.jpg
Downloading: https://cdn-p.cian.site/images/7/711/288/882117754-2.jpg
Downloading: https://cdn-p.cian.site/images/6/875/018/kvartira-moskva-pereulok-dokuchaev-810578622-2.jpg
Downloading: https://cdn-p.cian.site/images/8/852/798/kvartira-moskva-ulica-bolshaya-ordynka-897258897-2.jpg
Downloading: https://cdn-p.cian.site/images/2/811/119/kvartira-moskva-kutuzovskiy-prospekt-911118216-2.jpg
Downloading: https://cdn-p.cian.site/images/7/103/009/900301712-2.jpg
Downloading: https://cdn-p.cian.site/images/8/782/009/900287894-2.jpg
Downloading: https://cdn-p.cian.site/images/7/760/548/kvartira-moskva-smolenskaya-ulica-845067706-2.jpg
Downloading: https://cdn-p.cian.site/images/3/544/558/kvartira-moskva-kutuzovskiy-prospekt-855445342-2.jpg
Downloading: https://cdn-p.cian.site/images/1/069/188/kvartira-moskva-mytnaya-ulica-881960180-2.jpg
Downloading: https://cdn-p.cian.site/images/0/437/909/kvartira-moskva-horoshevskoe-shoss

Downloading: https://cdn-p.cian.site/images/9/534/038/kvartira-moskva-basmannyy-pereulok-830435955-2.jpg
Downloading: https://cdn-p.cian.site/images/9/053/204/kvartira-moskva-minskaya-ulica-402350961-2.jpg



In [ ]:
torch.load("../")

In [253]:
from pathlib import Path
from tqdm.auto import tqdm
import orjson

In [254]:
Path("flats_dict_avito.json").write_bytes(orjson.dumps(flats_dict_avito))
Path("flats_dict_cian.json").write_bytes(orjson.dumps(flats_dict_cian))

195638

In [256]:
!pip install ipyannotate

/bin/sh: pip: command not found


In [255]:
!du -d 1 -h .

 52M	./models
 20K	./__pycache__
644K	./.ipynb_checkpoints
1016K	./.git
162M	./data
710M	./tg_images
 80K	./.idea
928M	.


In [6]:
%%time
#2. Building model and predict one image

import cv2
import numpy as np
import io


# images 320x320

# effnet b0
# adaptive avg pool
# ? -> 512 -> relu -> bn -> dropout 0.5
# 512 -> 2, softmax 

# accuracy
# lr 1e-4

def _fast_expand(img):
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img


def pseudo_download_image(url):
#     print(f'[INFO] Downloading {url}')
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image

dict_keys(['https://www.cian.ru/rent/flat/235726043/', 'https://www.cian.ru/rent/flat/234507390/', 'https://www.cian.ru/rent/flat/235568526/', 'https://www.cian.ru/rent/flat/235591522/', 'https://www.cian.ru/rent/flat/235811492/', 'https://www.cian.ru/rent/flat/234828601/', 'https://www.cian.ru/rent/flat/235051812/', 'https://www.cian.ru/rent/flat/166979646/', 'https://www.cian.ru/rent/flat/233758211/', 'https://www.cian.ru/rent/flat/235821048/', 'https://www.cian.ru/rent/flat/235488071/', 'https://www.cian.ru/rent/flat/235733251/', 'https://www.cian.ru/rent/flat/235822495/', 'https://www.cian.ru/rent/flat/235567887/', 'https://www.cian.ru/rent/flat/235204532/', 'https://www.cian.ru/rent/flat/235868714/', 'https://www.cian.ru/rent/flat/235884878/', 'https://www.cian.ru/rent/flat/235912368/', 'https://www.cian.ru/rent/flat/234107006/', 'https://www.cian.ru/rent/flat/235943911/', 'https://www.cian.ru/rent/flat/199218370/', 'https://www.cian.ru/rent/flat/235569307/', 'https://www.cian.ru/

In [226]:
list(flats_dict)[0]

'1939075739'

Downloading: https://03.img.avito.st/image/1/ngFoTba_Muge6MDuWCOOCJ_uNOLWLjYa2u4w7tDoMOjcqA
Downloading: https://87.img.avito.st/image/1/JgVcfba_iuwq2HjqHBE2DKvejObiHo4e7t6I6uTYiOzomA
Downloading: https://56.img.avito.st/image/1/pRzB9ba_CfW3UPvzh5u1FTZWD_9_lg0Hc1YL83lQC_V1EA
Downloading: https://00.img.avito.st/image/1/1b1B7La_eVQ3SYtSZ5_FtLZPf17_j32m8097UvlJe1T1CQ
Downloading: https://31.img.avito.st/image/1/3gxcfba_cuUq2IDjbDbOBavedO_iHnYX7t5w4-TYcOXomA
Downloading: https://22.img.avito.st/image/1/x3hje7a_a5EV3pmXYQTvGJTYbZvdGG9j0dhpl9veaZHXng
Downloading: https://21.img.avito.st/image/1/PFqRCLa_kLPnrWK15XQUOmarlrkva5RBI6uStSmtkrMl7Q
Downloading: https://68.img.avito.st/image/1/zYn5jLa_YWCPKZNm78_l6Q4vZ2pH72WSSy9jZkEpY2BNaQ
Downloading: https://53.img.avito.st/image/1/3HwHhLa_cJVxIYKTM8f0HPAndp-553RntSdyk78hcpWzYQ
Downloading: https://79.img.avito.st/image/1/xqJTT7a_aksl6phNUwzuwqTsbEHtLG654exoTevqaEvnqg
Downloading: https://20.img.avito.st/image/1/XaqObLa_8UP4yQNFuF0CjHnP90kwD_WxPM_

Downloading: https://51.img.avito.st/image/1/CiLaIba_psushFTNwkUsakKCoMFkQqI5aIKkzWKEpMtuxA
Downloading: https://51.img.avito.st/image/1/CiLaIba_psushFTNwkUsakKCoMFkQqI5aIKkzWKEpMtuxA
Downloading: https://58.img.avito.st/image/1/AzKzMba_r9vFlF3duVUleiuSqdENUqspAZKt3QuUrdsH1A
Downloading: https://58.img.avito.st/image/1/AzKzMba_r9vFlF3duVUleiuSqdENUqspAZKt3QuUrdsH1A
Downloading: https://51.img.avito.st/image/1/aseZ7ba_xi7vSDQowegaz29OwCQnjsLcK07EKCFIxC4tCA
Downloading: https://51.img.avito.st/image/1/aseZ7ba_xi7vSDQowegaz29OwCQnjsLcK07EKCFIxC4tCA
Downloading: https://66.img.avito.st/image/1/61fu_ba_R76YWLW4tP6bXxheQbRQnkNMXF5FuFZYRb5aGA
Downloading: https://66.img.avito.st/image/1/61fu_ba_R76YWLW4tP6bXxheQbRQnkNMXF5FuFZYRb5aGA
Downloading: https://68.img.avito.st/image/1/9aTRhba_WU2nIKtLt_a0ziYmX0dv5l2_YyZbS2kgW01lYA
Downloading: https://68.img.avito.st/image/1/9aTRhba_WU2nIKtLt_a0ziYmX0dv5l2_YyZbS2kgW01lYA
Downloading: https://72.img.avito.st/image/1/EZrX2ba_vXOhfE91-d1hkiF6u3lpurmBZXq

Downloading: https://16.img.avito.st/image/1/Mcr6Lra_nSOMi28llHIE_2ONmylETZnRSI2fJUKLnyNOyw
Downloading: https://16.img.avito.st/image/1/Mcr6Lra_nSOMi28llHIE_2ONmylETZnRSI2fJUKLnyNOyw
Downloading: https://90.img.avito.st/image/1/D26El7a_o4fyMlGBlss6Wx00pY069Kd1NjShgTwyoYcwcg
Downloading: https://90.img.avito.st/image/1/D26El7a_o4fyMlGBlss6Wx00pY069Kd1NjShgTwyoYcwcg
Downloading: https://02.img.avito.st/image/1/5_evo7a_Sx7ZBrkY7fTksyQATRQRwE_sHQBJGBcGSR4bRg
Downloading: https://02.img.avito.st/image/1/5_evo7a_Sx7ZBrkY7fTksyQATRQRwE_sHQBJGBcGSR4bRg
Downloading: https://14.img.avito.st/image/1/myomora_N8NQB8XFHPSYbq0BMcmYwTMxlAE1xZ4HNcOSRw
Downloading: https://14.img.avito.st/image/1/myomora_N8NQB8XFHPSYbq0BMcmYwTMxlAE1xZ4HNcOSRw
Downloading: https://29.img.avito.st/image/1/iMu1ILa_JCLDhdYkqXaLjz6DIigLQyDQB4MmJA2FJiIBxQ
Downloading: https://29.img.avito.st/image/1/iMu1ILa_JCLDhdYkqXaLjz6DIigLQyDQB4MmJA2FJiIBxQ
Downloading: https://46.img.avito.st/image/1/u-p3M7a_FwMBluUFDWW4rvyQEQnJUBPxxZA

Downloading: https://17.img.avito.st/image/1/bDyjDLa_wNXVqTLT93UgRFevxt8db8QnEa_C0xupwtUX6Q
Downloading: https://17.img.avito.st/image/1/bDyjDLa_wNXVqTLT93UgRFevxt8db8QnEa_C0xupwtUX6Q
Downloading: https://66.img.avito.st/image/1/I1UO0La_j7x4dX26RPBYLfpzibaws4tOvHONurZ1jby6NQ
Downloading: https://66.img.avito.st/image/1/I1UO0La_j7x4dX26RPBYLfpzibaws4tOvHONurZ1jby6NQ
Downloading: https://50.img.avito.st/image/1/j2yPSLa_I4X57dGDnW70FHvrJY8xKyd3PeshgzftIYU7rQ
Downloading: https://50.img.avito.st/image/1/j2yPSLa_I4X57dGDnW70FHvrJY8xKyd3PeshgzftIYU7rQ
Downloading: https://51.img.avito.st/image/1/arePbLa_xl75yTRY10kRz3vPwFQxD8KsPc_EWDfJxF47iQ
Downloading: https://51.img.avito.st/image/1/arePbLa_xl75yTRY10kRz3vPwFQxD8KsPc_EWDfJxF47iQ
Downloading: https://48.img.avito.st/image/1/xfdJDba_aR4_qJsYTya-j72ubxT3bm3s-65rGPGoax796A
Downloading: https://48.img.avito.st/image/1/xfdJDba_aR4_qJsYTya-j72ubxT3bm3s-65rGPGoax796A
Downloading: https://80.img.avito.st/image/1/QRFXbLa_7fghyR_-WUU6aaPP6_LpD-kK5c_

Downloading: https://70.img.avito.st/image/1/f6xdM7a_00UrliFDL2pZ1KmQ1U_jUNe375DRQ-WW0UXp1g
Downloading: https://70.img.avito.st/image/1/f6xdM7a_00UrliFDL2pZ1KmQ1U_jUNe375DRQ-WW0UXp1g
Downloading: https://83.img.avito.st/image/1/crc0nLa_3l5COSxYXMVUz8A_2FSK_9qshj_cWIw53F6AeQ
Downloading: https://83.img.avito.st/image/1/crc0nLa_3l5COSxYXMVUz8A_2FSK_9qshj_cWIw53F6AeQ
Downloading: https://99.img.avito.st/image/1/YmmJ_7a_zoD_WjyGwaZEEX1cyIo3nMpyO1zMhjFazIA9Gg
Downloading: https://99.img.avito.st/image/1/YmmJ_7a_zoD_WjyGwaZEEX1cyIo3nMpyO1zMhjFazIA9Gg
Downloading: https://15.img.avito.st/image/1/EtTqIba_vj2chEw7wnk0rB6CuDdUQrrPWIK8O1KEvD1exA
Downloading: https://15.img.avito.st/image/1/EtTqIba_vj2chEw7wnk0rB6CuDdUQrrPWIK8O1KEvD1exA
Downloading: https://38.img.avito.st/image/1/O1Ky3ba_l7vEeGW9yIUdKkZ-kbEMvpNJAH6VvQp4lbsGOA
Downloading: https://38.img.avito.st/image/1/O1Ky3ba_l7vEeGW9yIUdKkZ-kbEMvpNJAH6VvQp4lbsGOA
Downloading: https://64.img.avito.st/image/1/eRFclra_1fgqMyf-Iu89aKg10_Li9dEK7jX

...

Downloading: https://03.img.avito.st/image/1/ohBZf7a_Dvkv2vz_EVG4aMzcCPPnHAoL69wM_-HaDPntmg
Downloading: https://03.img.avito.st/image/1/ohBZf7a_Dvkv2vz_EVG4aMzcCPPnHAoL69wM_-HaDPntmg
Downloading: https://67.img.avito.st/image/1/_iQPXra_Us15-6DLf23kXJr9VMexPVY_vf1Qy7f7UM27uw
Downloading: https://67.img.avito.st/image/1/_iQPXra_Us15-6DLf23kXJr9VMexPVY_vf1Qy7f7UM27uw
Downloading: https://89.img.avito.st/image/1/5ByIRLa_SPX-4brzzHf-ZB3nTv82J0wHOudK8zDhSvU8oQ
Downloading: https://89.img.avito.st/image/1/5ByIRLa_SPX-4brzzHf-ZB3nTv82J0wHOudK8zDhSvU8oQ
Downloading: https://62.img.avito.st/image/1/Z3TLWLa_y529_TmbiXV9DF77zZd1O89vefvJm3P9yZ1_vQ
Downloading: https://62.img.avito.st/image/1/Z3TLWLa_y529_TmbiXV9DF77zZd1O89vefvJm3P9yZ1_vQ
Downloading: https://21.img.avito.st/image/1/hAxBnba_KOU3ONrjRa-edNQ-Lu___iwX8z4q4_k4KuX1eA
Downloading: https://21.img.avito.st/image/1/hAxBnba_KOU3ONrjRa-edNQ-Lu___iwX8z4q4_k4KuX1eA
Downloading: https://13.img.avito.st/image/1/SGhLmra_5IE9PxaHV_MfAr854ov1-eBz-Tn

Downloading: https://67.img.avito.st/image/1/Kg5oiLa_huceLXThMKg_AZ4rgO3W64IV2iuE4dAthOfcbQ
Downloading: https://67.img.avito.st/image/1/Kg5oiLa_huceLXThMKg_AZ4rgO3W64IV2iuE4dAthOfcbQ
Downloading: https://42.img.avito.st/image/1/ny7KGra_M8e8v8HB-DyKITy5Nc10eTc1eLkxwXK_Mcd-_w
Downloading: https://42.img.avito.st/image/1/ny7KGra_M8e8v8HB-DyKITy5Nc10eTc1eLkxwXK_Mcd-_w
Downloading: https://07.img.avito.st/image/1/PsKZ3La_kivveWAt6fQi9DZ_lCEnv5bZK3-QLSF5kCstOQ
Downloading: https://07.img.avito.st/image/1/PsKZ3La_kivveWAt6fQi9DZ_lCEnv5bZK3-QLSF5kCstOQ
Downloading: https://12.img.avito.st/image/1/yXwKN7a_ZZV8kpeTFATVSqWUY5-0VGFnuJRnk7KSZ5W-0g
Downloading: https://12.img.avito.st/image/1/yXwKN7a_ZZV8kpeTFATVSqWUY5-0VGFnuJRnk7KSZ5W-0g
Downloading: https://20.img.avito.st/image/1/cXwKt7a_3ZV8Ei-TZIJtSqUU25-01NlnuBTfk7IS35W-Ug
Downloading: https://20.img.avito.st/image/1/cXwKt7a_3ZV8Ei-TZIJtSqUU25-01NlnuBTfk7IS35W-Ug
Downloading: https://13.img.avito.st/image/1/6GH9fba_RIiL2LaOoUL0V1LeQoJDHkB6T95

Downloading: https://70.img.avito.st/image/1/A3RrHba_r50duF2bYX0TH5--qZfVfqtv2b6tm9O4rZ3f-A
Downloading: https://70.img.avito.st/image/1/A3RrHba_r50duF2bYX0TH5--qZfVfqtv2b6tm9O4rZ3f-A
Downloading: https://91.img.avito.st/image/1/LhVbV7a_gvwt8nD6Czc-fq_0hPblNIYO6fSA-uPygPzvsg
Downloading: https://91.img.avito.st/image/1/LhVbV7a_gvwt8nD6Czc-fq_0hPblNIYO6fSA-uPygPzvsg
Downloading: https://12.img.avito.st/image/1/ldaM9ba_OT_6UMs5qr_P3i9WPzUylj3NPlY7OTRQOz84EA
Downloading: https://12.img.avito.st/image/1/ldaM9ba_OT_6UMs5qr_P3i9WPzUylj3NPlY7OTRQOz84EA
Downloading: https://19.img.avito.st/image/1/kpHCD7a_Pni0qsx-6kXImWGsOHJ8bDqKcKw8fnqqPHh26g
Downloading: https://19.img.avito.st/image/1/kpHCD7a_Pni0qsx-6kXImWGsOHJ8bDqKcKw8fnqqPHh26g
Downloading: https://24.img.avito.st/image/1/iTgTwLa_JdFlZdfXDYrTMLBjI9utoyEjoWMn16tlJ9GnJQ
Downloading: https://24.img.avito.st/image/1/iTgTwLa_JdFlZdfXDYrTMLBjI9utoyEjoWMn16tlJ9GnJQ
Downloading: https://30.img.avito.st/image/1/h6UjLLa_K0xVidlKIWbdrYCPLUadTy--kY8

Downloading: https://20.img.avito.st/image/1/eZ40ira_1XdCLydxSpc8isMp032K6dGFhinXcYwv13eAbw
Downloading: https://24.img.avito.st/image/1/mWmf1ra_NYDpc8eGocjcfWh1M4ohtTFyLXU3hidzN4ArMw
Downloading: https://80.img.avito.st/image/1/ocLDIba_DSu1hP8tjT_k1jSCCyF9QgnZcYIPLXuEDyt3xA
Downloading: https://46.img.avito.st/image/1/S-WoPLa_5wzemRUKsilvjlyf4QYWX-P-Gp_lChCZ5Qwc2Q
Downloading: https://83.img.avito.st/image/1/BmIVeLa_qotj3ViNFUB_XK7brIGrG655p9uoja3dqIuhnQ
Downloading: https://97.img.avito.st/image/1/yINDwba_ZGo1ZJZsX8KxvfhiYmD9omCY8WJmbPtkZmr3JA
Downloading: https://73.img.avito.st/image/1/3P2QLLa_cBTmiYISpG-lwyuPdh4uT3TmIo9yEiiJchQkyQ
Downloading: https://09.img.avito.st/image/1/aINHgba_xGoxJDZsG8QRvfwiwmD54sCY9SLGbP8kxmrzZA
Downloading: https://28.img.avito.st/image/1/daao_7a_2U_eWitJzoYMmBNc30UWnN29GlzbSRBa208cGg
Downloading: https://60.img.avito.st/image/1/qc8AL7a_BSZ2ivcgXk2F74mMAyy-TAHUsowHILiKBya0yg
Downloading: https://60.img.avito.st/image/1/qc8AL7a_BSZ2ivcgXk2F74mMAyy-TAHUsow

KeyboardInterrupt: 

In [ ]:
### MAIN PART

PAGE_SIZE = 28


def find_and_classify(cian_url, threshold=0.5):
    result_scores = {}
    flats_dict = {}
    search_results_count = get_search_results_count(cian_url)
    if not search_results_count:
        return

    print(f'There are {search_results_count} flats at your request. Grabbing the data...')

    max_page_number = search_results_count // PAGE_SIZE
    if not search_results_count % PAGE_SIZE or max_page_number == 0:
        max_page_number += 1

    for page in range(1, max_page_number + 1):
        url = f'{cian_url}&p={page}'
        flats_dict.update(load_flat_dicts(url))

    print(f"{len(flats_dict.keys())} flats with pictures found. Let's drop the shit...")

    for f, urls in flats_dict.items():
        data = {'score': [0, 0], 'thumb': urls[0]}
        total_score = [0, 0]
        for url in urls:
            data['score'] = predict_image(url)
            total_score = [a + b for a, b in zip(total_score, data['score'])]

        final_score = [x/len(urls) for x in total_score]
        data['score'] = final_score
        result_scores[f] = data
        classes = ['bad', 'good']
        print(f'Score for {f} -> {final_score}: This is a {classes[final_score.index(max(final_score))]} flat.')

    #ПОКАЗАТЬ ТОЛЬКО ОТСЕЯННЫЕ КВАРТИРЫ. threshold – порог качество от 0 до 1
    if result_scores:
        result_scores = {
            k: v for k, v in sorted(
              result_scores.items(),
              key=lambda item: max(item[1]['score']),
              reverse=True,
            ) if is_good(v['score'], threshold)
        }
        print(f'\n{len(result_scores.keys())} perfect flats found!\n')
        show_results(result_scores, threshold=threshold)
